In [1]:
import pytest
import numpy as np
from modules.utils.services import embedder_services

# ==========================================
# 🧩 1️⃣ Kiểm tra Dense Embedding
# ==========================================
def test_encode_dense_basic():
    text = "Phân tích thị trường chứng khoán Việt Nam"
    dense_vecs = embedder_services.encode_dense(text)
    assert isinstance(dense_vecs, list), "Dense output phải là list"
    assert isinstance(dense_vecs[0], list), "Mỗi vector phải là list"
    assert len(dense_vecs[0]) == 384, f"Chiều dense vector phải = 384, hiện tại = {len(dense_vecs[0])}"
    print("\n✅ [Dense] OK - Chiều vector:", len(dense_vecs[0]))
    print("🔹 First 5 dims:", dense_vecs[0][:5])


# ==========================================
# 🧩 2️⃣ Kiểm tra Fit BM25
# ==========================================
def test_fit_bm25_and_vocab():
    corpus = [
        "Vingroup hỗ trợ đồng bào miền Trung",
        "Thị trường chứng khoán Việt Nam biến động",
        "Giá cổ phiếu VCB tăng mạnh hôm nay",
        "Techcombank và Vietcombank công bố kết quả kinh doanh quý 3"
    ]

    embedder_services.fit_bm25(corpus)
    assert embedder_services.bm25 is not None, "BM25 chưa được fit"
    assert len(embedder_services.vocab) > 0, "Từ vựng BM25 rỗng"
    print("\n✅ [BM25] OK - Vocab size:", len(embedder_services.vocab))


# ==========================================
# 🧩 3️⃣ Kiểm tra Sparse Embedding
# ==========================================
def test_encode_sparse_after_fit():
    text = "Phân tích cổ phiếu VCB"
    sparse_vecs = embedder_services.encode_sparse(text)
    assert isinstance(sparse_vecs, list), "Sparse output phải là list"
    assert "indices" in sparse_vecs[0] and "values" in sparse_vecs[0], "Cấu trúc sparse không hợp lệ"
    print("\n✅ [Sparse] OK - Số token trong vector:", len(sparse_vecs[0]['indices']))
    print("🔹 Sample indices:", sparse_vecs[0]['indices'][:5])
    print("🔹 Sample values:", sparse_vecs[0]['values'][:5])


# ==========================================
# 🧩 4️⃣ Kiểm tra Dense + Sparse song song
# ==========================================
def test_hybrid_encode():
    text = "Giá cổ phiếu VCB tăng mạnh"
    dense = embedder_services.encode_dense(text)
    sparse = embedder_services.encode_sparse(text)

    assert len(dense[0]) == 384, "Dense vector phải có 384 chiều"
    assert isinstance(sparse[0]['indices'], list), "Sparse indices phải là list"
    print("\n✅ [Hybrid] OK - Dense(384) + Sparse(length=", len(sparse[0]['indices']), ")")
    print("🔹 Dense first dims:", dense[0][:5])


# ==========================================
# 🧩 5️⃣ Kiểm tra lỗi nếu chưa fit BM25
# ==========================================
def test_sparse_before_fit():
    # Tạo instance mới chưa fit để test
    from modules.utils.services import EmbedderServices
    new_embedder = EmbedderServices(auto_fit=False)
    with pytest.raises(ValueError, match="BM25 chưa được fit"):
        new_embedder.encode_sparse("cổ phiếu VCB")
    print("\n✅ [Error Handling] OK - Cảnh báo chưa fit BM25 hoạt động đúng")
if __name__ == "__main__":
    test_encode_dense_basic()
    test_fit_bm25_and_vocab()
    test_encode_sparse_after_fit()
    test_hybrid_encode()
    test_sparse_before_fit()

c:\Users\admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
W1015 08:51:00.512000 1456 site-packages\torch\distributed\elastic\multiprocessing\redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.
Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


Collection `cafef_articles` đã tồn tại.
EmbedderServices device: cuda
RerankerServices device: cuda

✅ [Dense] OK - Chiều vector: 384
🔹 First 5 dims: [-0.42621153593063354, 0.4696739912033081, -0.08442647755146027, -0.06147570535540581, -0.23391616344451904]

✅ [BM25] OK - Vocab size: 34

✅ [Sparse] OK - Số token trong vector: 3
🔹 Sample indices: [6, 15, 25]
🔹 Sample values: [0.8472978603872037, 0.8472978603872037, 0.8472978603872037]

✅ [Hybrid] OK - Dense(384) + Sparse(length= 6 )
🔹 Dense first dims: [-0.3635924756526947, 0.16256432235240936, -0.22329771518707275, 0.07123153656721115, -0.05664300546050072]
EmbedderServices device: cuda

✅ [Error Handling] OK - Cảnh báo chưa fit BM25 hoạt động đúng


In [1]:
from modules.core.state import GlobalState
from modules.nodes.embedder import embed_query
import numpy as np

def test_embedder():
    print("=== 🧩 TEST EMBEDDER MODULE (rag & hybrid) ===")

    for route_type in ["rag", "hybrid"]:
        print(f"\n🔹 TEST ROUTE: {route_type.upper()}")
        state = GlobalState(user_query="Hôm nay có tin tức gì mới?", route_to=route_type)

        state = embed_query(state)

        if state.query_embedding is None:
            print("❌ Không tạo được embedding.")
            print("Debug info:", state.debug_info)
            continue

        dense_vec = state.query_embedding.get("dense_vector")
        sparse_vec = state.query_embedding.get("sparse_vector")

        print("✅ Embedding thành công.")
        print(f"🧠 Dense vector length: {len(dense_vec) if dense_vec is not None else 'None'}")
        if isinstance(dense_vec, (list, np.ndarray)):
            print(f"🔹 First 5 dims: {dense_vec[:5]}")
        else:
            print("⚠️ Dense vector không phải list hoặc np.ndarray")

        print(f"🌾 Sparse vector: {'OK' if sparse_vec else 'None'}")
        print(f"📜 Debug Info: {state.debug_info}")
        print("LLM status:", state.llm_status)
        print("Embed route:", state.debug_info.get('embed_route'))

if __name__ == "__main__":
    test_embedder()


c:\Users\admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
W1015 13:48:54.003000 24268 site-packages\torch\distributed\elastic\multiprocessing\redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.
Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


Collection `cafef_articles` đã tồn tại.


`torch_dtype` is deprecated! Use `dtype` instead!
Device set to use cuda:0


EmbedderServices device: cuda
RerankerServices device: cuda
=== 🧩 TEST EMBEDDER MODULE (rag & hybrid) ===

🔹 TEST ROUTE: RAG
✅ Embedding thành công.
🧠 Dense vector length: 384
🔹 First 5 dims: [-0.6698017120361328, 0.020612990483641624, -0.15784049034118652, 0.1114264577627182, -0.6839015483856201]
🌾 Sparse vector: OK
📜 Debug Info: {'embed_dense_dim': 384, 'embed_sparse_status': 'ok', 'embed_status': 'success', 'embed_query_text': 'Hôm nay có tin tức gì mới?', 'embed_route': 'rag'}
LLM status: embed_success
Embed route: rag

🔹 TEST ROUTE: HYBRID
✅ Embedding thành công.
🧠 Dense vector length: 384
🔹 First 5 dims: [-0.6698017120361328, 0.020612990483641624, -0.15784049034118652, 0.1114264577627182, -0.6839015483856201]
🌾 Sparse vector: OK
📜 Debug Info: {'embed_dense_dim': 384, 'embed_sparse_status': 'ok', 'embed_status': 'success', 'embed_query_text': 'Hôm nay có tin tức gì mới?', 'embed_route': 'hybrid'}
LLM status: embed_success
Embed route: hybrid


In [1]:
from modules.core.state import GlobalState
from modules.nodes.processor import processor_query   # 🧠 Gọi Processor để detect intent + time_filter
from modules.nodes.embedder import embed_query
from modules.nodes.vector_db import search_vector_db
from datetime import datetime
import pytz

def test_vector_db_realdata():
    """
    🧩 TEST VECTOR_DB TRÊN DỮ LIỆU THẬT (CAFÉF)
    ------------------------------------------------
    ✅ Gọi Processor để detect intent + time_filter
    ✅ Kiểm tra embedding + hybrid search + reranker
    ✅ In chi tiết debug + khoảng thời gian lọc
    """
    print("\n=== 🧩 TEST VECTOR_DB TRÊN DỮ LIỆU THẬT (CAFÉF) ===")

    query = "Phân tích cổ phiểu VCB hôm nay?"
    state = GlobalState(user_query=query, debug=True)

    # --- Bước 1: Processor (detect intent + time_filter + tickers) ---
    state = processor_query(state)

    vn_tz = pytz.timezone("Asia/Ho_Chi_Minh")
    print(f"\n🧠 Query gốc: {state.user_query}")
    print(f"🗣️ Intent: {getattr(state, 'intent', 'unknown')}")
    print(f"⏰ Time Filter: {state.time_filter}")
    if state.time_filter:
        start_ts, end_ts = state.time_filter
        print(f"   → {datetime.fromtimestamp(start_ts, vn_tz)} → {datetime.fromtimestamp(end_ts, vn_tz)}")
    else:
        print("   → Không phát hiện mốc thời gian, sẽ dùng mặc định 3 ngày gần nhất.")

    # --- Chạy test cho 2 route: RAG và HYBRID ---
    for route in ["rag", "hybrid"]:
        print(f"\n🔹 TEST ROUTE: {route.upper()}")
        state.route_to = route

        try:
            # --- Bước 2: EMBEDDING ---
            state = embed_query(state)
            if not state.query_embedding:
                print("❌ Không tạo được embedding.")
                print("Debug info:", getattr(state, "debug_info", {}))
                continue

            dense_dim = len(state.query_embedding.get("dense_vector", []))
            print(f"✅ [Dense] OK - Chiều vector: {dense_dim}")

            sparse_vec = state.query_embedding.get("sparse_vector")
            if sparse_vec:
                print(f"✅ [Sparse] OK - Số token trong vector: {len(sparse_vec[0]['indices'])}")
            else:
                print("⚠️ [Sparse] Không có sparse vector (BM25 chưa fit hoặc corpus rỗng).")

            # --- Bước 3: SEARCH (Hybrid + Reranker) ---
            state = search_vector_db(state, top_k=5, alpha=0.7)

            # --- Kiểm tra reranker ---
            rerank_status = getattr(state, "debug_info", {}).get("reranker_status", "unknown")
            if "applied" in rerank_status:
                print(f"✅ [Reranker] ĐÃ ÁP DỤNG — Model: {state.debug_info.get('reranker_model')} | Device: {state.debug_info.get('reranker_device')}")
            elif "failed" in rerank_status:
                print(f"❌ [Reranker] LỖI: {rerank_status}")
            else:
                print(f"⚠️ [Reranker] BỎ QUA — {rerank_status}")

            # --- Bước 4: In kết quả ---
            print(f"\n=== 🔍 KẾT QUẢ TÌM KIẾM CHO TRUY VẤN: “{query}” ===")
            if not state.search_results:
                print("⚠️ Không tìm thấy kết quả nào trong Qdrant.")
            else:
                for idx, doc in enumerate(state.search_results, 1):
                    title = doc.get("title", "❌ Không có tiêu đề")
                    time = doc.get("time", "❌ Không rõ thời gian")
                    snippet = doc.get("content", "") or "(Không có nội dung trích dẫn)"
                    print(f"{idx}. [{doc.get('score', 0):.4f}] {title} — {time}")
                    print(f"   {snippet}\n")

            # --- Debug tổng kết ---
            print("📜 Debug Info:", getattr(state, "debug_info", {}))
            print("🧠 LLM Status:", getattr(state, "llm_status", "undefined"))

        except Exception as e:
            print(f"❌ Lỗi trong route {route}: {e}")

    print("\n✅ [Test] Hoàn tất kiểm thử Vector DB + Reranker.\n")


if __name__ == "__main__":
    test_vector_db_realdata()


c:\Users\admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\vnai\scope\profile.py:562: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
W1015 14:55:36.740000 3128 site-packages\torch\distributed\elastic\multiprocessing\redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.
Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


Collection `cafef_articles` đã tồn tại.
EmbedderServices device: cuda
RerankerServices device: cuda

=== 🧩 TEST VECTOR_DB TRÊN DỮ LIỆU THẬT (CAFÉF) ===

🧠 Query gốc: Phân tích cổ phiểu VCB hôm nay?
🗣️ Intent: stock
⏰ Time Filter: (1760461200, 1760547599)
   → 2025-10-15 00:00:00+07:00 → 2025-10-15 23:59:59+07:00

🔹 TEST ROUTE: RAG
✅ [Dense] OK - Chiều vector: 384
✅ [Sparse] OK - Số token trong vector: 5
[TimeFilter] ⏰ Phát hiện khoảng thời gian trong query: 2025-10-15 00:00:00+07:00 → 2025-10-15 23:59:59+07:00
[FusionSearch] Found 10 results
Ngân hàng MB đăng ký mua 45,76 triệu cổ phiếu chào bán của MBS | score=0.6 | time_ts=1760495880
Vingroup lập kỷ lục chưa từng có trong lịch sử VN, tỷ phú Phạm Nhật Vượng tạo kỳ tích trong cùng 1 ngày | score=0.75 | time_ts=1760496300
Ngân hàng MB đăng ký mua 45,76 triệu cổ phiếu chào bán của MBS | score=0.625 | time_ts=1760495880
✅ [Reranker] ĐÃ ÁP DỤNG — Model: cross-encoder/ms-marco-MiniLM-L-6-v2 | Device: cuda

=== 🔍 KẾT QUẢ TÌM KIẾM CHO TRUY VẤ

In [2]:
from modules.utils.services import qdrant_services
res, _ = qdrant_services.client.scroll(collection_name="cafef_articles", limit=50)
for hit in res:
    print(hit.payload.get("title"), hit.payload.get("time"))


Chỉ số chứng khoán quan trọng của Việt Nam lần đầu vượt 2.000 điểm 13-10-2025 15:39:00
Góc nhìn chuyên gia: Dòng tiền lớn nhập cuộc, VN-Index vẫn còn tiềm năng tăng thêm 20–30% 12-10-2025 17:10:00
Công nghệ Việt tiến vào thị trường Mỹ: U2U Network được Kraken thông báo niêm yết 13-10-2025 15:42:00
Hình ảnh của Shark Bình bị tạm giam 14-10-2025 15:53:00
VPBankS gặp gỡ hơn 50 đối tác quốc tế, sẵn sàng cho thương vụ IPO lịch sử 14-10-2025 09:13:00
Bitcoin tăng chóng mặt, chuyên gia cảnh báo rủi ro 'tiền ảo vô giá trị' 11-10-2025 13:48:00
Phó TGĐ VinaCapital: Nâng hạng giúp tăng chiều sâu cho thị trường chứng khoán, hạn chế sự "áp đảo" của NĐT cá nhân 14-10-2025 10:27:00
Người nhà lãnh đạo SeABank muốn bán 5 triệu cổ phiếu SSB 12-10-2025 10:25:00
Bitcoin tăng chóng mặt, chuyên gia cảnh báo rủi ro 'tiền ảo vô giá trị' 11-10-2025 13:48:00
Phó TGĐ VinaCapital: Nâng hạng giúp tăng chiều sâu cho thị trường chứng khoán, hạn chế sự "áp đảo" của NĐT cá nhân 14-10-2025 10:27:00
Chứng khoán DNSE chu

In [1]:
from modules.core.state import GlobalState
from modules.nodes.processor import processor_query
from modules.nodes.router import route_intent
from modules.nodes.embedder import embed_query
from modules.nodes.vector_db import search_vector_db
from modules.nodes.retriever import retrieve_documents
from modules.api import stock_api
from datetime import datetime
import pytz


def test_rag_pipeline_realdata():
    """
    🧩 TEST FULL RAG PIPELINE TRÊN DỮ LIỆU THẬT (CAFÉF)
    -------------------------------------------------------
    Chuỗi xử lý:
    Processor → Router → Embedder → VectorDB → Retriever
    - Tự động phát hiện intent và time_filter
    - In log chi tiết, giúp debug từng giai đoạn
    """

    print("\n=== 🧩 TEST FULL RAG PIPELINE TRÊN DỮ LIỆU THẬT ===")

    # 1️⃣ Khởi tạo GlobalState
    state = GlobalState(
        user_query="Phân tích cổ phiếu VCB hôm nay?",
        debug=True
    )
    print(f"\n🧠 [Input Query] {state.user_query}")

    # 2️⃣ Processor — xử lý truy vấn người dùng
    state = processor_query(state)
    print(f"✅ [Processor] processed_query = {state.processed_query}")
    print(f"   intent = {state.intent}, time_filter = {state.time_filter}")
    print(f"   tickers = {state.tickers}")

    # 3️⃣ Router — xác định hướng xử lý (api, rag, hybrid)
    state = route_intent(state)
    print(f"➡️ [Router] route_to = {state.route_to}, api_type = {getattr(state, 'api_type', None)}")

    # 4️⃣ Nếu query là dạng API (chứng khoán, thời tiết, v.v.)
    if getattr(state, "api_type", None) == "stock":
        symbol = state.tickers[0] if state.tickers else "VCB"
        state.api_response = stock_api.format_stock_info(symbol)
        print(f"📊 [API] Stock response for {symbol}: {state.api_response}")
        print("\n✅ [Pipeline Completed via API route]\n")
        return

    # 5️⃣ Embedder — tạo dense + sparse embedding
    state = embed_query(state)
    if not getattr(state, "query_embedding", None):
        print("❌ [Embedder] Không tạo được embedding.")
        return

    dense_dim = len(state.query_embedding.get("dense_vector", []))
    sparse_vector = state.query_embedding.get("sparse_vector")
    sparse_len = len(sparse_vector[0]["indices"]) if sparse_vector else 0
    print(f"✅ [Embedder] Dense dim = {dense_dim}, Sparse tokens = {sparse_len}")

    # 6️⃣ Vector DB — tìm kiếm từ Qdrant (Hybrid dense + sparse)
    state = search_vector_db(state, top_k=5, alpha=0.7)
    print(f"🔍 [VectorDB] Found {len(state.search_results)} results | llm_status={state.llm_status}")

    if not getattr(state, "search_results", None):
        print("⚠️ [VectorDB] Không có tài liệu nào trong Qdrant.")
        return

    # 7️⃣ Retriever — hợp nhất chunk, gắn nhãn thời gian, tạo context
    state = retrieve_documents(state, max_chars=2000)
    print(f"📄 [Retriever] Retrieved {len(state.retrieved_docs)} docs, context_len={len(state.context)}")

    # Hiển thị trước một phần context (tối đa 400 ký tự)
    preview = state.context[:400].replace("\n", " ")
    print(f"🧩 [Context Preview] {preview}...")

    # 8️⃣ Tổng kết
    vn_tz = pytz.timezone("Asia/Ho_Chi_Minh")
    now = datetime.now(vn_tz).strftime("%d-%m-%Y %H:%M:%S")
    print(f"\n🕒 [Run completed at] {now}")
    print(f"🧠 [LLM Status] {state.llm_status}")
    print("✅ [Test] Hoàn tất kiểm thử full RAG pipeline.\n")


if __name__ == "__main__":
    test_rag_pipeline_realdata()


c:\Users\admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\vnai\scope\profile.py:562: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
W1015 15:03:49.066000 16216 site-packages\torch\distributed\elastic\multiprocessing\redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.
Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


Collection `cafef_articles` đã tồn tại.
EmbedderServices device: cuda
RerankerServices device: cuda

=== 🧩 TEST FULL RAG PIPELINE TRÊN DỮ LIỆU THẬT ===

🧠 [Input Query] Phân tích cổ phiếu VCB hôm nay?
✅ [Processor] processed_query = phân tích cổ phiếu vcb hôm nay
   intent = stock, time_filter = (1760461200, 1760547599)
   tickers = ['VCB']
➡️ [Router] route_to = hybrid, api_type = stock_news
✅ [Embedder] Dense dim = 384, Sparse tokens = 6
[TimeFilter] ⏰ Phát hiện khoảng thời gian trong query: 2025-10-15 00:00:00+07:00 → 2025-10-15 23:59:59+07:00
[FusionSearch] Found 10 results
Ngân hàng MB đăng ký mua 45,76 triệu cổ phiếu chào bán của MBS | score=0.6 | time_ts=1760495880
Vingroup lập kỷ lục chưa từng có trong lịch sử VN, tỷ phú Phạm Nhật Vượng tạo kỳ tích trong cùng 1 ngày | score=0.75 | time_ts=1760496300
Ngân hàng MB đăng ký mua 45,76 triệu cổ phiếu chào bán của MBS | score=0.625 | time_ts=1760495880
🔍 [VectorDB] Found 10 results | llm_status=vector_db_success
*** RETRIEVED DOCS ***

In [3]:
from modules.utils.services import qdrant_services

points, _ = qdrant_services.client.scroll(
    collection_name="cafef_articles", 
    with_payload=True, 
    limit=5
)
for p in points:
    print(p.payload)


{'id': '0015a8ef4bd041daf8af9acd1247785f', 'title': 'Góc nhìn chuyên gia: Dòng tiền lớn nhập cuộc, VN-Index vẫn còn tiềm năng tăng thêm 20–30%', 'time': '12-10-2025 17:10:00', 'time_ts': 1760263800, 'summary': 'Chuyên gia khuyến nghị nhà đầu tư không nên mua đuổi bằng mọi giá, mà nên áp dụng\xa0chiến lược giải ngân theo từng phần.', 'url': 'https://cafef.vn/goc-nhin-chuyen-gia-dong-tien-lon-nhap-cuoc-vn-index-van-con-tiem-nang-tang-them-2030-188251012163930025.chn', 'content': 'nâng hạng. Đây sẽ là tín hiệu khởi đầu cho một giai đoạn mới của thị trường chứng khoán Việt Nam ,ông nhận định. Theo ông Hoàng, độ bền của xu hướng tăng sẽ phụ thuộc vào sự cộng hưởng của ba trụ cột: Thanh khoản – Lợi nhuận doanh nghiệp – Yếu tố vĩ mô. Thị trường hiện đang ở giai đoạn hai của chu kỳ tăng – giai đoạn bứt phá ( Public Participation theo Lý thuyết Dow). Trong giai đoạn này, thanh khoản dồi dào là yếu tố tiên quyết. Dòng vốn ngoại theo câu chuyện nâng hạng sẽ tiếp tục là động lực chính trong 6–12 t

In [2]:
from modules.core.state import GlobalState
from modules.nodes.processor import processor_query
from modules.nodes.router import route_intent
from modules.nodes.embedder import embed_query
from modules.nodes.vector_db import search_vector_db
from modules.nodes.retriever import retrieve_documents
from modules.nodes.prompt_builder import build_prompt
from modules.api import stock_api
from datetime import datetime
import pytz

def test_prompt_realdata():
    """
    🧩 TEST PROMPT BUILDER TRÊN DỮ LIỆU THẬT
    ------------------------------------------------
    🔹 Pipeline: Processor → Router → Embedder → VectorDB → Retriever → Prompt
    🔹 Mục tiêu: kiểm tra prompt có đủ context, time_filter, API, và conversation history
    """
    print("\n=== 🧩 TEST PROMPT BUILDER TRÊN DỮ LIỆU THẬT ===\n")

    # 1️⃣ Khởi tạo state
    state = GlobalState()
    state.debug = True
    state.user_query = "Hôm nay có tin tức gì mới?"

    # 2️⃣ Processor
    state = processor_query(state)
    print(f"🧠 [Processor] processed_query = {state.processed_query}")
    if getattr(state, "time_filter", None):
        start_ts, end_ts = state.time_filter
        vn_tz = pytz.timezone("Asia/Ho_Chi_Minh")
        print(f"⏰ [TimeFilter] {datetime.fromtimestamp(start_ts, vn_tz)} → {datetime.fromtimestamp(end_ts, vn_tz)}")

    # 3️⃣ Router
    state = route_intent(state)
    print(f"➡️ [Router] route_to = {state.route_to}, api_type = {state.api_type}")

    # 4️⃣ API nếu cần
    if getattr(state, "api_type", None) == "stock":
        state.api_response = stock_api.format_stock_info("TCB")
        print(f"📊 [API] response = {state.api_response[:120]}...")

    # 5️⃣ Embedder
    state = embed_query(state)
    dense_dim = len(state.query_embedding.get("dense_vector", [])) if state.query_embedding else 0
    sparse_len = len(state.query_embedding.get("sparse_vector", [{}])[0].get("indices", [])) if state.query_embedding else 0
    print(f"🧩 [Embedder] Dense dim = {dense_dim}, Sparse tokens = {sparse_len}")

    # 6️⃣ Vector DB Search
    state = search_vector_db(state, top_k=5, alpha=0.7)
    print(f"🔍 [VectorDB] Found {len(state.search_results)} results | llm_status={state.llm_status}")

    # 7️⃣ Retriever
    state = retrieve_documents(state, max_chars=1500)
    print(f"📄 [Retriever] Retrieved {len(state.retrieved_docs)} docs | context_len={len(state.context)}")

    # 8️⃣ Build Prompt
    state = build_prompt(state, max_context_chars=1200)
    print(f"🧠 [Prompt] length={len(state.prompt)}, docs={len(state.retrieved_docs)}")

    # 9️⃣ In preview prompt
    print("\n=== 🔍 PROMPT SAMPLE (first 600 chars) ===")
    print(state.prompt[:600])
    print("---------------------------------------------")
    print(f"🧠 LLM status: {getattr(state, 'llm_status', 'undefined')}")

    print("\n✅ [Test] Hoàn tất kiểm thử PROMPT BUILDER.\n")


if __name__ == "__main__":
    test_prompt_realdata()



=== 🧩 TEST PROMPT BUILDER TRÊN DỮ LIỆU THẬT ===

🧠 [Processor] processed_query = hôm nay có tin tức gì mới
⏰ [TimeFilter] 2025-10-15 00:00:00+07:00 → 2025-10-15 23:59:59+07:00
➡️ [Router] route_to = rag, api_type = news
🧩 [Embedder] Dense dim = 384, Sparse tokens = 7
[TimeFilter] ⏰ Phát hiện khoảng thời gian trong query: 2025-10-15 00:00:00+07:00 → 2025-10-15 23:59:59+07:00
[FusionSearch] Found 9 results
Vingroup lập kỷ lục chưa từng có trong lịch sử VN, tỷ phú Phạm Nhật Vượng tạo kỳ tích trong cùng 1 ngày | score=0.625 | time_ts=1760496300
Vingroup lập kỷ lục chưa từng có trong lịch sử VN, tỷ phú Phạm Nhật Vượng tạo kỳ tích trong cùng 1 ngày | score=0.75 | time_ts=1760496300
Vingroup lập kỷ lục chưa từng có trong lịch sử VN, tỷ phú Phạm Nhật Vượng tạo kỳ tích trong cùng 1 ngày | score=0.75 | time_ts=1760496300
🔍 [VectorDB] Found 9 results | llm_status=vector_db_success
*** RETRIEVED DOCS ***
{'id': '33e80e10-b5aa-9ff0-0cc3-34d5e46329f0', 'score': 0.767, 'title': 'Tín hiệu tạo đỉnh ng

In [5]:
from modules.nodes.response_generator import response_node  # import file chứa response_node
from modules.core.state import GlobalState
from modules.nodes.processor import processor_query
from modules.nodes.router import route_intent
from modules.api import stock_api
from modules.nodes.embedder import embed_query
from modules.nodes.vector_db import search_vector_db
from modules.nodes.retriever import retrieve_documents
from modules.nodes.prompt_builder import build_prompt


def test_full_rag_pipeline_realdata():
    """
    🧩 TEST FULL RAG PIPELINE TRÊN DỮ LIỆU THẬT (END-TO-END)
    ----------------------------------------------------------
    Pipeline:
      user_query → processor → router → api → embedder → vector_db
      → retriever → prompt_builder → response_node (LLM)
    ✅ Kiểm tra toàn bộ luồng RAG thật từ query đến phản hồi.
    """
    print("\n=== 🧩 TEST FULL RAG PIPELINE TRÊN DỮ LIỆU THẬT ===")

    # 1️⃣ Khởi tạo state
    state = GlobalState(user_query="Tổng quan thị trường chứng khoán hôm qua?", debug=True)

    # 2️⃣ Processor
    state = processor_query(state)
    print(f"🧠 [Processor] query_clean = {state.processed_query}")

    # 3️⃣ Router
    state = route_intent(state)
    print(f"🧭 [Router] route_to = {state.route_to}, api_type = {state.api_type}")

    # 4️⃣ API (khi cần)
    # ⚙️ Sửa ở đây: bao gồm cả stock và stock_news (hybrid)
    if getattr(state, "api_type", None) in ["stock", "stock_news"]:
        try:
            state.api_response = stock_api.format_stock_info("VCB")
            print(f"📊 [API] response = {state.api_response[:250]}...")
        except Exception as e:
            print(f"⚠️ [API] Lỗi khi lấy dữ liệu VCB: {e}")

    # 5️⃣ Embedder
    state = embed_query(state)
    dense_dim = len(state.query_embedding['dense_vector']) if state.query_embedding else 0
    print(f"🧩 [Embedder] Dense dim = {dense_dim}")

    # 6️⃣ Vector DB
    state = search_vector_db(state, top_k=10, alpha=0.7)
    print(f"🔍 [VectorDB] Found {len(state.search_results)} results (llm_status={state.llm_status})")

    # 7️⃣ Retriever
    state = retrieve_documents(state, max_chars=1500)
    print(f"📄 [Retriever] Retrieved {len(state.retrieved_docs)} docs, context_len={len(state.context)}")

    print("=== CHECK RETRIEVER OUTPUT ===")
    print(f"context length = {len(state.context)}")
    print(f"retrieved_docs = {len(state.retrieved_docs)}")
    print("Preview context:")
    print(state.context[:1200])

    # 8️⃣ Prompt Builder
    state = build_prompt(state, max_context_chars=1200)
    print(f"📜 [Prompt] Built successfully, length={len(state.prompt)}")

    # 9️⃣ Response Generator (LLM)
    state = response_node(state)
    print(f"🤖 [LLM] Response length={len(state.final_answer)}")
    print(f"🧾 [Final Answer]:\n{state.final_answer[:800]}")

    # 10️⃣ Debug Info Summary
    print("\n=== 🧩 DEBUG SUMMARY ===")
    for k, v in getattr(state, "debug_info", {}).items():
        print(f"{k}: {v}")
    print("✅ [Test] Hoàn tất kiểm thử full RAG pipeline.\n")


if __name__ == "__main__":
    test_full_rag_pipeline_realdata()


2025-10-16 09:19:41,128 [DEBUG] httpcore.connection: connect_tcp.started host='localhost' port=6333 local_address=None timeout=5.0 socket_options=None


2025-10-16 09:19:41,131 [DEBUG] httpcore.connection: connect_tcp.complete return_value=<httpcore._backends.sync.SyncStream object at 0x000001453470C550>
2025-10-16 09:19:41,132 [DEBUG] httpcore.http11: send_request_headers.started request=<Request [b'POST']>
2025-10-16 09:19:41,134 [DEBUG] httpcore.http11: send_request_headers.complete
2025-10-16 09:19:41,135 [DEBUG] httpcore.http11: send_request_body.started request=<Request [b'POST']>
2025-10-16 09:19:41,137 [DEBUG] httpcore.http11: send_request_body.complete
2025-10-16 09:19:41,138 [DEBUG] httpcore.http11: receive_response_headers.started request=<Request [b'POST']>
2025-10-16 09:19:41,142 [DEBUG] httpcore.http11: receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'transfer-encoding', b'chunked'), (b'content-encoding', b'br'), (b'content-type', b'application/json'), (b'vary', b'accept-encoding, Origin, Access-Control-Request-Method, Access-Control-Request-Headers'), (b'date', b'Thu, 16 Oct 2025 02:19:40 GMT'


=== 🧩 TEST FULL RAG PIPELINE TRÊN DỮ LIỆU THẬT ===
🧠 [Processor] query_clean = tổng quan thị trường chứng khoán hôm qua
🧭 [Router] route_to = hybrid, api_type = market_analysis
🧩 [Embedder] Dense dim = 384


Batches: 100%|██████████| 1/1 [00:00<00:00, 43.85it/s]


[FusionSearch] Found 10 results
Kết thúc Quý 3, DNSE hoàn thành vượt mức mục tiêu lợi nhuận cả năm 2025 | score=0.625 | time_ts=1760524020
Kết thúc Quý 3, DNSE hoàn thành vượt mức mục tiêu lợi nhuận cả năm 2025 | score=0.6 | time_ts=1760524020
SHS đạt 1.379 tỷ đồng lợi nhuận sau 9 tháng, hoàn thành 100,7% kế hoạch năm 2025 | score=0.5955 | time_ts=1760524200
SHS đạt 1.379 tỷ đồng lợi nhuận sau 9 tháng, hoàn thành 100,7% kế hoạch năm 2025 | score=0.6625 | time_ts=1760524200
336 triệu cổ phiếu chứng khoán nhận tin vui, thị giá "bốc đầu" kịch trần | score=0.6667 | time_ts=1760516040
🔍 [VectorDB] Found 10 results (llm_status=vector_db_success)
*** RETRIEVED DOCS ***
{'id': '097b9721-066a-136b-9b56-2bd4bc7458db', 'score': 0.75, 'title': 'Chứng khoán ngày 16-10: Cổ phiếu ngân hàng, bất động sản… dẫn dắt dòng tiền?', 'time': '15-10-2025 22:47:00', 'time_ts': 1760543220, 'url': '', 'content_preview': 'DIG, NVL có mức tăng giá đáng chú ý. Kết thúc phiên giao dịch, VN- Index đóng cửa tại 1.757 đ

2025-10-16 09:19:41,517 [DEBUG] openai._base_client: Request options: {'method': 'post', 'url': '/chat/completions', 'headers': {'X-Stainless-Raw-Response': 'true'}, 'files': None, 'idempotency_key': 'stainless-python-retry-29d609c0-bb96-4e4f-9829-93e012e9958e', 'json_data': {'messages': [{'content': "- Luôn trả lời bằng TIẾNG VIỆT nếu có từ ngữ chuyên ngành (VNINDEX, VN30,..) thì giữ nguyên.\n\n## Instruction:\nBạn là **trợ lý AI tài chính Việt Nam**, chuyên phân tích xu hướng thị trường, cổ phiếu và tin tức.\n1. Khi intent = 'market' thì KẾT HỢP dữ liệu API (giá cổ phiếu, thị trường) với tin tức (Context).\n2. Khi intent = 'stock' thì Trình bày thông tin giá cổ phiếu ngắn gọn.\n3. KHÔNG khuyến nghị đầu tư tuyệt đối.\n## Constraints:\n- Luôn trả lời bằng TIẾNG VIỆT, ngắn gọn, tự nhiên.\n- KHÔNG dùng mã code, KHÔNG in cấu trúc JSON.\n- KHÔNG nói “Tôi không phải chuyên gia tài chính”.\n- Giữ nguyên các ký hiệu như VNINDEX, VCB, VN30,...\n## Bối cảnh thời gian:\nHôm nay là Thứ Năm, Ngày 

📄 [Retriever] Retrieved 7 docs, context_len=2516
=== CHECK RETRIEVER OUTPUT ===
context length = 2516
retrieved_docs = 7
Preview context:
[Tin tức cập nhập đến: Hôm nay là Thứ Năm, Ngày 16 tháng 10 năm 2025.
Ngày hôm qua là Ngày 15 tháng 10 năm 2025.
Ngày mai là Ngày 17 tháng 10 năm 2025.
Tuần sau sẽ bắt đầu từ Ngày 23 tháng 10 năm 2025.
Tuần trước bắt đầu từ Ngày 09 tháng 10 năm 2025.]

[Chứng khoán ngày 16-10: Cổ phiếu ngân hàng, bất động sản… dẫn dắt dòng tiền? | 15-10-2025 22:47:00] (score=0.75, rerank=2.3688)
DIG, NVL có mức tăng giá đáng chú ý. Kết thúc phiên giao dịch, VN- Index đóng cửa tại 1.757 điểm, giảm 3 điểm (tương đương 0,18%) Theo nhận định của Công ty Chứng khoán VCBS, VN-Index đang trong giai đoạn củng cố động lực quanh vùng 1.750-1.780 điểm. Sự phân hóa giữa các nhóm cổ phiếu blue-chip cho thấy thị trường đang điều chỉnh sau đợt tăng giá mạnh trước đó. VCBS khuyến nghị nhà đầu tư cân nhắc chốt lời ngắn hạn đối với các mã đã đạt mục tiêu hoặc có tín hiệu đảo chiều, đồ

2025-10-16 09:19:53,018 [DEBUG] httpcore.http11: receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Content-Length', b'7426'), (b'Content-Type', b'application/json'), (b'Date', b'Thu, 16 Oct 2025 02:19:41 GMT'), (b'Ngrok-Agent-Ips', b'117.2.56.232'), (b'Server', b'uvicorn')])
2025-10-16 09:19:53,020 [INFO] httpx: HTTP Request: POST https://nonomissible-winfred-doggedly.ngrok-free.dev/v1/chat/completions "HTTP/1.1 200 OK"
2025-10-16 09:19:53,021 [DEBUG] httpcore.http11: receive_response_body.started request=<Request [b'POST']>
2025-10-16 09:19:53,022 [DEBUG] httpcore.http11: receive_response_body.complete
2025-10-16 09:19:53,023 [DEBUG] httpcore.http11: response_closed.started
2025-10-16 09:19:53,023 [DEBUG] httpcore.http11: response_closed.complete
2025-10-16 09:19:53,024 [DEBUG] openai._base_client: HTTP Response: POST https://nonomissible-winfred-doggedly.ngrok-free.dev/v1/chat/completions "200 OK" Headers({'content-length': '7426', 'content-type': 'applicat

🤖 [LLM] Response length=368
🧾 [Final Answer]:
VNINDEX đóng cửa phiên 15/10 tại 1.757 điểm, giảm 3 điểm (0,18%). Thị trường có sự phân hóa mạnh, với nhóm ngân hàng và bất động sản dẫn dắt dòng tiền. DIG và NVL ghi nhận mức tăng đáng chú ý. Theo VCBS, VNINDEX đang trong giai đoạn củng cố quanh vùng 1.750-1.780 điểm. Trong khi đó, SHS công bố lợi nhuận sau 9 tháng đạt 1.379 tỷ đồng, hoàn thành 100,7% kế hoạch năm.

=== 🧩 DEBUG SUMMARY ===
route: hybrid
embed_dense_dim: 384
embed_sparse_status: ok
embed_status: success
embed_query_text: tổng quan thị trường chứng khoán hôm qua
embed_route: hybrid
reranker_status: applied
vector_db_results: 10
prompt_intent: market
prompt_status: built_success
llm_status: response_generated
intent: market
timestamp: 2025-10-16T09:19:53.027637
✅ [Test] Hoàn tất kiểm thử full RAG pipeline.

